## Creating Pages for EEGAtlas

In [1]:
from collections import defaultdict
import os
import json

In [2]:
categories = ['normal-eeg','normalvariants-eeg','artifacts','abnormal-eeg','icu-eeg']

category_clean = {'normal-eeg': "Normal EEG", 'abnormal-eeg':'Abnormal EEG',
                     "icu-eeg":"ICU EEG", "artifacts":"Artifacts", "normalvariants-eeg":"Normal variants EEG"}

## add invasive-eeg 

In [3]:
eeg_atlas_file = os.path.abspath('../_data/eegatlas.txt')

def read_eegatlas_textfile(path_to_eegatlas_file):
    result = {} # result dictionary
    
    f_split = open(path_to_eegatlas_file,'r').read().split('#')
    f_split = [x for x in f_split if x!='']
    
    #looping over each topic i.e normal-eeg, abnormal-eeg, artifact
    for ff in f_split: 
        lines = ff.split('\n')
        category = lines[0]
        eegs = [x for x in lines[1:] if x!='']
        result[category]=eegs
    return result

In [4]:
eeg_atlas = read_eegatlas_textfile(eeg_atlas_file)
eeg_atlas

{'normal-eeg': ['Awake, adult PDR ',
  'Awake, pediatric PDR',
  'Drowsiness, slowing of PDR ',
  'Stage 2 sleep, sleep spindles ',
  'Stage 2 sleep, K complexes',
  'Drowsiness, diffuse irregular delta-theta slowing of background',
  'Stage 1/2 sleep, vertex waves',
  'Drowsiness, slow roving lateral eye movements ',
  'Stage 1/2 sleep, POSTS ',
  'Stage 3 sleep, diffuse irregular delta slowing ',
  'Rapid eye movement (REM) sleep, erratic eye movements ',
  'Rapid eye movement (REM) sleep, sawtooth waves '],
 'normalvariants-eeg': ['wicket spike/rhythm',
  'Hyperventilation-induced slowing ',
  'benign epileptiform transients of sleep',
  'Breach rhythm',
  'Rhythmic mid-temporal theta of drowsiness ',
  'Hypnopompic hypersynchrony ',
  'Hypnagogic hypersynchrony ',
  'Photic driving ',
  'Mu rhythm ',
  'Posterior slow waves of youth ',
  'Photomyogenic response ',
  'Lambda waves ',
  'Temporal slowing of the elderly ',
  '6 Hz spike-and-wave bursts ',
  'Frontal arousal rhythm',
 

In [11]:
class EEGPage(object):
    Template = """---
layout: post
date: 2022-08-02 
title: {title}
subtitle: {subtitle}
author: 
permalink: /EEGAtlas/{category}/{eeg}
---
""" 
    category_clean = {'normal-eeg': "Normal EEG", 'abnormal-eeg':'Abnormal EEG',
                     "icu-eeg":"ICU EEG", "artifacts":"Artifacts", "normalvariants-eeg":"Normal variants EEG"}
    
    def __init__(self, eeg_category, eeg): 
        self.eeg_category = eeg_category
        self.eeg_oldname = eeg
        self.eeg = self._clean_eeg_name(eeg)
        self.eeg_dir = self._create_eeg_dir()
        self.eeg_link = '/EEGAtlas/{category}/{eeg}'.format(category=category,eeg=self.eeg)
        self.eeg_image_dir = self._create_eeg_image_dir()
        
    def _clean_eeg_name(self,eeg):
        file_name = eeg
        file_name = file_name.strip()
        file_name = file_name.replace(' ','_')
        file_name = file_name.replace('/','_')
        file_name= file_name.replace('<','_less_than')
        file_name = file_name.replace(',','_')
        file_name = file_name.replace('__','_')
        
        return file_name
    
    def _create_eeg_dir(self):
        
        eeg_dir = os.path.abspath('../EEGAtlas/{category}/{eeg}'.format(category=self.eeg_category,eeg=self.eeg))
        
        try:
            os.makedirs(eeg_dir)
        except FileExistsError:
            pass
        
        return eeg_dir

    
    def _create_eeg_image_dir(self):
        img_dir = '../assets/img/EEGAtlas/{category}/{eeg}'.format(category=self.eeg_category,eeg=self.eeg)
        try:
            os.makedirs(img_dir)
        except FileExistsError:
            pass
        return img_dir 
    
    def get_examples(self):
        examples = [x for x in os.listdir(self.eeg_image_dir) if 'example' in x]
        return examples
        
    def get_example_caption(self,example):
        metadata_path = os.path.join(self.eeg_image_dir,example,'metadata.json')
        f = open(metadata_path)
        data = json.load(f)
        return data['Caption']
    
    def add_example(self,example):
        result = ''
        strip_path = os.path.join('/assets/img/EEGAtlas/{category}/{eeg}/{example}'.format(category=self.eeg_category,
                                                                                           eeg=self.eeg,
                                                                                           example=example),'strip.png')
        caption = self.get_example_caption(example)
        result +='![{example}]({strip_path})'.format(example=example,strip_path=strip_path)
        result+='\n'
        result+='_{caption}_'.format(caption=caption)
        return result
        
    
    def write_eegpage(self,overwrite=False):
        self.eegpage_file = os.path.join(self.eeg_dir,'{eeg}.md'.format(eeg=self.eeg))
        if not os.path.exists(self.eegpage_file) or overwrite: 
            f = open(self.eegpage_file,'w')    
            page = EEGPage.Template.format(title = "{eeg}".format(eeg = eeg),subtitle=EEGPage.category_clean[category],
                                        category=category,eeg=self.eeg)
            page+='\n\n'
            
            #add examples
            examples = self.get_examples()
            for example in examples: 
                page+=self.add_example(example)
            
            f.write(page)
    
    

In [12]:
### create EEG pages
storage_dict = defaultdict(dict)

for category in categories:
    for eeg in eeg_atlas[category]:
        eegpage = EEGPage(category,eeg)
        eegpage.write_eegpage(overwrite=True)
        storage_dict[category][eeg] = defaultdict()
        storage_dict[category][eeg]['path'] = eegpage.eeg_link

### Create the EEGAtlas Post

In [13]:
storage_dict

defaultdict(dict,
            {'normal-eeg': {'Awake, adult PDR ': defaultdict(None,
                          {'path': '/EEGAtlas/normal-eeg/Awake_adult_PDR'}),
              'Awake, pediatric PDR': defaultdict(None,
                          {'path': '/EEGAtlas/normal-eeg/Awake_pediatric_PDR'}),
              'Drowsiness, slowing of PDR ': defaultdict(None,
                          {'path': '/EEGAtlas/normal-eeg/Drowsiness_slowing_of_PDR'}),
              'Stage 2 sleep, sleep spindles ': defaultdict(None,
                          {'path': '/EEGAtlas/normal-eeg/Stage_2_sleep_sleep_spindles'}),
              'Stage 2 sleep, K complexes': defaultdict(None,
                          {'path': '/EEGAtlas/normal-eeg/Stage_2_sleep_K_complexes'}),
              'Drowsiness, diffuse irregular delta-theta slowing of background': defaultdict(None,
                          {'path': '/EEGAtlas/normal-eeg/Drowsiness_diffuse_irregular_delta-theta_slowing_of_background'}),
              'Stage 1/

In [14]:
eegatlas_tab_template = """---
layout: eegatlas
title: EEG Atlas
date: 2022-08-02
toc: true
permalink: /EEGAtlas
---

"""

for category in categories:
    eegatlas_tab_template += " <h2> {category} </h2>\n".format(category = category_clean[category])
    for eeg in eeg_atlas[category]:
        eegatlas_tab_template += "<li> <a href='{path}'> {eeg} </a> \n".format(path=storage_dict[category][eeg]['path'],
                                                                             eeg = eeg)


In [15]:
f = open('../EEGAtlas/EEGAtlas.md','w')
f.write(eegatlas_tab_template)

8055